This note book prepare the data so that the data is ready as inputs to machine learning models.

In [29]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
cd /content/drive/MyDrive/JOB-HUNTING/CV_NOW/Accenture_REPL_DS/exercise

/content/drive/MyDrive/JOB-HUNTING/CV_NOW/Accenture_REPL_DS/exercise


# Data loading

In [32]:
sales = pd.read_csv('data/explored_sales.csv')
hourly_sales = pd.read_csv('data/hourly_sales.csv')
weekly_sales = pd.read_csv('data/weekly_sales.csv')
monthly_sales = pd.read_csv('data/monthly_sales.csv')


In [33]:
hourly_sales.head()

,hour,Value,num_of_hours,num_of_weeks
0,2013-11-07 13:00:00,3457.0,0,0
1,2013-11-07 14:00:00,2250.0,1,0
2,2013-11-07 15:00:00,0.0,2,0
3,2013-11-07 16:00:00,729.0,3,0
4,2013-11-07 17:00:00,0.0,4,0


In [34]:
print(min(hourly_sales.hour))
print(max(hourly_sales.hour))

2013-11-07 13:00:00
2018-05-06 19:00:00


type conversion: convert string to datetime

In [35]:
sales.hour = pd.to_datetime(sales.hour)
sales.Date = pd.to_datetime(sales.Date)
hourly_sales.hour = pd.to_datetime(hourly_sales.hour)



this is for the convenience of constructing featuers based on hours

In [36]:
hourly_sales.head()

,hour,Value,num_of_hours,num_of_weeks
0,2013-11-07 13:00:00,3457.0,0,0
1,2013-11-07 14:00:00,2250.0,1,0
2,2013-11-07 15:00:00,0.0,2,0
3,2013-11-07 16:00:00,729.0,3,0
4,2013-11-07 17:00:00,0.0,4,0


In [37]:
# this is for the convenience of constructing featuers based on hours
aux_hourly_sales = hourly_sales.copy(deep = True)
aux_hourly_sales = aux_hourly_sales[['hour','Value','num_of_hours']]
aux_hourly_sales.head()

,hour,Value,num_of_hours
0,2013-11-07 13:00:00,3457.0,0
1,2013-11-07 14:00:00,2250.0,1
2,2013-11-07 15:00:00,0.0,2
3,2013-11-07 16:00:00,729.0,3
4,2013-11-07 17:00:00,0.0,4


# Initial missing value imputation

# Construct Model Features

I will contruct a few different types of features:
- features accounting for time series temporal correlation
- feastures accounting for seasonality 
- features accounting for possible trend
- features accounting for outliers, e.g., holidays and naitonal days

Note that features for temporal correlation may also help to explain the seanality of the data, this is because when choosing what data from previous days, week, month, or year, we considered selectively choose data corresponding to seasons.

## 1. Time series temporal correlation

I will use index based row accesses when contruct features because the time complexity is O(1) using index based access.

### Long range AR (auto regresssive)

Since we are going to use data from about 1 year ago as features, data in the 1st year will not be used as training data with labels. Let's calculate the index for the 1st entry we are going to use as training data.

In [38]:
temp = hourly_sales.hour < hourly_sales.hour.iloc[0] + pd.Timedelta(373, "d")
print('Number of rows not used for training: {}'.format([sum(temp)]))
first_train_idx = sum(temp)

Number of rows not used for training: [8952]


In [39]:
hourly_sales.head()

,hour,Value,num_of_hours,num_of_weeks
0,2013-11-07 13:00:00,3457.0,0,0
1,2013-11-07 14:00:00,2250.0,1,0
2,2013-11-07 15:00:00,0.0,2,0
3,2013-11-07 16:00:00,729.0,3,0
4,2013-11-07 17:00:00,0.0,4,0


In [40]:
weekly_sales.head()

,num_of_weeks,Value
0,0,223513.0
1,1,215608.0
2,2,228981.0
3,3,216698.0
4,4,272244.0


In [41]:
# --(long range)the same time from last year
# weekly total 52 weeks (1 year) ago 
hourly_sales['num_of_weeks_52weeks_ago'] = hourly_sales.num_of_weeks-52
hourly_sales = hourly_sales.merge(weekly_sales, how = 'left', left_on = ['num_of_weeks_52weeks_ago'], right_on = ['num_of_weeks'])
hourly_sales = hourly_sales[['hour','Value_x','num_of_hours','num_of_weeks_x','Value_y']]
hourly_sales.rename(columns = {'Value_x': 'sales', 'num_of_weeks_x': 'num_of_weeks', 'Value_y': 'AR__sales_weekly_total_52weeks_ago'}, inplace = True)
# weekly total 51 weeksago 
hourly_sales['num_of_weeks_51weeks_ago'] = hourly_sales.num_of_weeks-51
hourly_sales = hourly_sales.merge(weekly_sales, how = 'left', left_on = ['num_of_weeks_51weeks_ago'], right_on = ['num_of_weeks'])
hourly_sales = hourly_sales[['hour','num_of_hours','num_of_weeks_x','sales','AR__sales_weekly_total_52weeks_ago','Value']]
hourly_sales.rename(columns = {'num_of_weeks_x': 'num_of_weeks','Value':'AR__sales_weekly_total_51weeks_ago'}, inplace = True)

#same hod value 364 days ago
from utility import add_long_range_hod
hourly_sales = add_long_range_hod(hourly_sales,aux_hourly_sales,num_days = 364 )
print(hourly_sales.shape)
hourly_sales.head()



(39391, 7)


,hour,num_of_hours,num_of_weeks,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_364_DAYS_ago_same_hod
0,2013-11-07 13:00:00,0,0,3457.0,NaN,NaN,NaN
1,2013-11-07 14:00:00,1,0,2250.0,NaN,NaN,NaN
2,2013-11-07 15:00:00,2,0,0.0,NaN,NaN,NaN
3,2013-11-07 16:00:00,3,0,729.0,NaN,NaN,NaN
4,2013-11-07 17:00:00,4,0,0.0,NaN,NaN,NaN


In [42]:
# this is for when we do not use long range AR features
hourly_sales.rename(columns = {'Value': 'sales'},inplace = True)

### Mid range AR

We add features in the mid range, such as the hourly sale in the same hour-of-day time segments (+/- 1 hour) 1-6 weeks ago. 

Notice that if we predict hourly sales/traffic 1 month ahead, some of the features in this category maybe from predicted, not from the truth.

In [43]:
from utility import add_mid_range_features
for num_weeks in range(5,8): # everything : range(1,7)   no data from within 1 month range(5,7)
  hourly_sales = add_mid_range_features(hourly_sales,aux_hourly_sales,num_weeks)
print(hourly_sales.shape)
hourly_sales.head()

(39391, 9)


,hour,num_of_hours,num_of_weeks,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_364_DAYS_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h
0,2013-11-07 13:00:00,0,0,3457.0,NaN,NaN,NaN,NaN,NaN
1,2013-11-07 14:00:00,1,0,2250.0,NaN,NaN,NaN,NaN,NaN
2,2013-11-07 15:00:00,2,0,0.0,NaN,NaN,NaN,NaN,NaN
3,2013-11-07 16:00:00,3,0,729.0,NaN,NaN,NaN,NaN,NaN
4,2013-11-07 17:00:00,4,0,0.0,NaN,NaN,NaN,NaN,NaN


### Short range AR

According to PACF for hourly sale data. I will use the following 2 types of features:
1. the hourly sales from the past 6 days in the same time-of-day segments (+/- 1 hour). 
2. hourly sales from the last 24 hours.

Notice that if we predict hourly sales/traffic 1 month ahead, Most of the time, the features in this category maybe from predicted, not from the truth.


In [44]:
# # ---the same time from the last 6 days
# from utility import add_short_range_features
# for num_days in range(1,2): # orginal: range(1,7)
#   hourly_sales = add_short_range_features(hourly_sales,aux_hourly_sales,num_days)
# hourly_sales.head()
# print(hourly_sales.shape)

# # ---the same time-of-day from last 22 hour data
# from utility import add_micro_range_features
# hourly_sales = add_micro_range_features(hourly_sales,aux_hourly_sales)
# print(hourly_sales.shape)
# hourly_sales.head()

## Season features (directly related to seasons)

In this section, categorical features related to natual seasons are added, such as hour of day, day of week, month

In [45]:
hourly_sales['seasonal__hod'] = hourly_sales.hour.dt.hour
hourly_sales['seasonal__dow'] = hourly_sales.hour.dt.dayofweek
# hourly_sales['seasonal__month'] = hourly_sales.hour.dt.month
hourly_sales.head()

,hour,num_of_hours,num_of_weeks,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_364_DAYS_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h,seasonal__hod,seasonal__dow
0,2013-11-07 13:00:00,0,0,3457.0,NaN,NaN,NaN,NaN,NaN,13,3
1,2013-11-07 14:00:00,1,0,2250.0,NaN,NaN,NaN,NaN,NaN,14,3
2,2013-11-07 15:00:00,2,0,0.0,NaN,NaN,NaN,NaN,NaN,15,3
3,2013-11-07 16:00:00,3,0,729.0,NaN,NaN,NaN,NaN,NaN,16,3
4,2013-11-07 17:00:00,4,0,0.0,NaN,NaN,NaN,NaN,NaN,17,3


## Holiday & naitonal days

From time series plot, we saw some outliers and some local in the time series around holiday or national days. In this section, I added features related to obvious outliers and local trends  associated with Thanksgiving, Christmas, Easter. The features here may not be a thourough solution. There could be more features to add in the future.

### Easter

No particular pattern was found around Easter,so just added a single boolean feature indicating if the date is Easter or not. Data from here (https://www.census.gov/data/software/x13as/genhol/easter-dates.html#par_textimage_1067001717)

In [46]:
Easters_list = ['2013-03-31','2014-04-20','2015-04-05','2016-03-27','2017-04-16','2018-04-01'] # easters from 2013 to 2018
Easter_set = set()
for item in Easters_list:
  Easter_set.add(pd.to_datetime(item))
hourly_sales['holiday__is_easter'] = hourly_sales.hour.dt.floor('d').apply(lambda x: x in Easter_set)

### Thanksgiving

In [47]:
from utility import calc_thanksgiving_same_year, calc_christmas_same_year, calc_next_immediate_christmas

In [48]:
thanksgiving_list = ['2013-11-28','2014-11-27','2015-11-26','2016-11-24','2017-11-23','2018-11-22'] 
thanksgiving_set = set()
for item in thanksgiving_list:
  thanksgiving_set.add(pd.to_datetime(item))
hourly_sales['holiday__is_thanksgiving'] = hourly_sales.hour.dt.floor('d').apply(lambda x: x in thanksgiving_set)
hourly_sales['holiday__is_thanksgiving_post_1d'] = hourly_sales.hour.apply(lambda x: (x.floor('d')-calc_thanksgiving_same_year(x))/pd.Timedelta(1,'d')==1)
hourly_sales['holiday__is_thanksgiving_post_2d'] = hourly_sales.hour.apply(lambda x: (x.floor('d')-calc_thanksgiving_same_year(x))/pd.Timedelta(1,'d')==2)
hourly_sales['holiday__is_thanksgiving_post_3d'] = hourly_sales.hour.apply(lambda x: (x.floor('d')-calc_thanksgiving_same_year(x))/pd.Timedelta(1,'d')==3)

### Christmas

In [49]:

hourly_sales.hour.dt.floor('d').iloc[0]

Timestamp('2013-11-07 00:00:00')

In [50]:
hourly_sales['holiday__is_christmas'] = hourly_sales.hour.apply(lambda x: x.month == 12 and x.day==25)
hourly_sales['holiday__is_christmas_post_1d'] = hourly_sales.hour.apply(lambda x: (x.floor('d') -  calc_christmas_same_year(x))/pd.Timedelta(1,'d')==1 )
# features below is because regular seasonal features may not work the best during between thanksgiven and christmas
hourly_sales['holiday__between_thanksgiving_n_christmas'] = hourly_sales.hour.apply(lambda x: (x.floor('d')> calc_thanksgiving_same_year(x)) and (x.floor('d')< calc_christmas_same_year(x))  )
hourly_sales['holiday__days_to_next_christmas'] = hourly_sales.hour.apply(lambda x: (calc_next_immediate_christmas(x) - x.floor('d'))/pd.Timedelta(1,'d')   )
# the following is added based on intial results
# hourly_sales['holiday__is_christmas_pre_1d'] = hourly_sales.hour.apply(lambda x: (x.floor('d') -  calc_christmas_same_year(x))/pd.Timedelta(1,'d')==-1 )
# hourly_sales['holiday__is_christmas_pre_2d'] = hourly_sales.hour.apply(lambda x: (x.floor('d') -  calc_christmas_same_year(x))/pd.Timedelta(1,'d')==-2 )
# hourly_sales['holiday__is_christmas_pre_3d'] = hourly_sales.hour.apply(lambda x: (x.floor('d') -  calc_christmas_same_year(x))/pd.Timedelta(1,'d')==-3 )

In [51]:
hourly_sales.head()

,hour,num_of_hours,num_of_weeks,sales,AR__sales_weekly_total_52weeks_ago,AR__sales_weekly_total_51weeks_ago,AR__sales_364_DAYS_ago_same_hod,AR__sales_5_weeks_ago_p_0h,AR__sales_6_weeks_ago_p_0h,seasonal__hod,seasonal__dow,holiday__is_easter,holiday__is_thanksgiving,holiday__is_thanksgiving_post_1d,holiday__is_thanksgiving_post_2d,holiday__is_thanksgiving_post_3d,holiday__is_christmas,holiday__is_christmas_post_1d,holiday__between_thanksgiving_n_christmas,holiday__days_to_next_christmas
0,2013-11-07 13:00:00,0,0,3457.0,NaN,NaN,NaN,NaN,NaN,13,3,False,False,False,False,False,False,False,False,48.0
1,2013-11-07 14:00:00,1,0,2250.0,NaN,NaN,NaN,NaN,NaN,14,3,False,False,False,False,False,False,False,False,48.0
2,2013-11-07 15:00:00,2,0,0.0,NaN,NaN,NaN,NaN,NaN,15,3,False,False,False,False,False,False,False,False,48.0
3,2013-11-07 16:00:00,3,0,729.0,NaN,NaN,NaN,NaN,NaN,16,3,False,False,False,False,False,False,False,False,48.0
4,2013-11-07 17:00:00,4,0,0.0,NaN,NaN,NaN,NaN,NaN,17,3,False,False,False,False,False,False,False,False,48.0


## feature for trend

Not sure we have yearly trend. Maybe we can use monthly/weekly/dayly trend?

# Save featurized data

In [52]:
print(min(hourly_sales.hour))
print(max(hourly_sales.hour))

2013-11-07 13:00:00
2018-05-06 19:00:00


In [53]:
hourly_sales.to_csv('data/hourly_sales_features.csv',index=False)

In [54]:
hourly_sales.shape

(39391, 20)

In [55]:
hourly_sales.hour.iloc[0]

Timestamp('2013-11-07 13:00:00')

In [56]:
hourly_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39391 entries, 0 to 39390
Data columns (total 20 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   hour                                       39391 non-null  datetime64[ns]
 1   num_of_hours                               39391 non-null  int64         
 2   num_of_weeks                               39391 non-null  int64         
 3   sales                                      39391 non-null  float64       
 4   AR__sales_weekly_total_52weeks_ago         30668 non-null  float64       
 5   AR__sales_weekly_total_51weeks_ago         30836 non-null  float64       
 6   AR__sales_364_DAYS_ago_same_hod            30655 non-null  float64       
 7   AR__sales_5_weeks_ago_p_0h                 38551 non-null  float64       
 8   AR__sales_6_weeks_ago_p_0h                 38383 non-null  float64       
 9   seasonal__hod    